<a href="https://colab.research.google.com/github/simi12345/Sentiment-Analysis/blob/main/sample_SNAtokp**.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nlp-id

     |████████████████████████████████| 7.5MB 5.4MB/s 
     |████████████████████████████████| 7.0MB 42.0MB/s 
     |████████████████████████████████| 1.5MB 36.7MB/s 
  Created wheel for nlp-id: filename=nlp_id-0.1.10.0-cp37-none-any.whl size=7723181 sha256=82733d687209720800206f0c7e5507c85b544011230e90eee22ec766954b3d11
  Stored in directory: /root/.cache/pip/wheels/a5/b6/2b/146e3f2a36d470240782b5b54021d994c974aaefeee5561e77
  Created wheel for nltk: filename=nltk-3.4.5-cp37-none-any.whl size=1449923 sha256=3b00cbc593990dd38971a137eaba05dab60a2294501bc9042725b13b34095b47
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=7d7456fc951313c099eeddd3a33a2f934538af24f7a6f5e1016af98e6ee5b9e9
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built nlp-id nltk wget
  Found existing installati

In [ ]:
!pip install emoji

     |████████████████████████████████| 133kB 6.9MB/s 


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

KeyboardInterrupt: ignored

In [ ]:
import nltk
import random
from sklearn.model_selection import train_test_split
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re
import csv
from nltk.corpus import stopwords
import string
import emoji

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
with open('dataset/dataset_tokped.csv', 'r', encoding="utf8") as nodecsv: # Buka file
    csvreader = csv.reader(nodecsv) # membaca data
    # Menyusun data dalam list dan menghilangkan header data
    csv = [n for n in csvreader][1:]

In [ ]:
print(csv[:10])

[['ZM Zaskia Mecca - Makassar Sand Scarf Kerudung Segi Empat Wangi Water', 'bagus', 'positive', 'bagus'], ['ZM Zaskia Mecca - Cendana Black Scarf - Romansa Khatuliatiwa', 'kerudung nya bagus .', 'positive', 'bagus'], ['ZM Zaskia Mecca - Yogya Charcoal Scarf Kerudung Segi Empat', 'barang dan harga ok', 'positive', 'bagus'], ['ZM Zaskia Mecca - Jola Plum Scarf Kerudung Segi Empat Wangi - All Size', 'lembut dan adem', 'positive', 'bagus'], ['ZM - Gyan Wine Dress Jelita Indonesia Edisi Palembang - M', 'barang sesuai.. terimakasihh', 'positive', 'bagus'], ['ZM Zaskia Mecca - Lamun Ombak Dove Scarf Kerudung Segi Empat', 'Barang original, bahannya bagus. Keren banget dapat harga flash sale 👍🏻👍🏻👍🏻', 'positive', 'bagus'], ['ZM Zaskia Mecca - Burung Merak Dusty Blue Scarf Kerudung Segi Empat', 'fast respon oke', 'positive', 'bagus'], ['ZM Zaskia Mecca - Kelimutu Stone Scarf Kerudung Segi Empat Ultra Sonic', 'selalu suka produk2 ZM', 'positive', 'bagus'], ['ZM Zaskia Mecca - Sana Khaky Hijab Keru

In [ ]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
regex_str = []
regex_str.append(emoticons_str)
regex_str.append(r'<[^>]+>')# HTML tags
regex_str.append(r'(?:@[\w_]+)')# @-mentions
regex_str.append(r'(?:&[\w_]+)')
regex_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
regex_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
regex_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers
regex_str.append(r"(?:[a-z][a-z'\-_]+[a-z])") # words with - and '
regex_str.append(r'(?:[\w_]+)') # other words
regex_str.append(r'(?:\S)') # anything else

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nlp_id.stopword import StopWord
stopword = StopWord()

In [ ]:
punctuation = list(string.punctuation)
stop = stopword.get_stopword() + punctuation + ['rt', 'via', '…','•','“']

In [ ]:
def tokenize(s):
    tokens = tokens_re.findall(s)
    return tokens

def cleanTweet(token,regex):
    terms_all = [emoji.demojize(term) for term in token if term.lower() not in stop and not regex.match(term)]
    return terms_all

In [ ]:
tokens = []
for c in csv:
    tokens.append(tokenize(c[1]))

print(tokens[:5])

[['bagus'], ['kerudung', 'nya', 'bagus', '.'], ['barang', 'dan', 'harga', 'ok'], ['lembut', 'dan', 'adem'], ['barang', 'sesuai', '.', '.', 'terimakasihh']]


In [ ]:
import operator

exclude_str = []
exclude_str.append(emoticons_str)
exclude_str.append(r'<[^>]+>')# HTML tags
exclude_str.append(r'(?:@[\w_]+)')# @-mentions
exclude_str.append(r'(?:&[\w_]+)')
exclude_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
exclude_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
exclude_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers

exclude_re = re.compile(r'('+'|'.join(exclude_str)+')', re.VERBOSE | re.IGNORECASE)

In [ ]:
all_word=[]
from collections import Counter
count_all = Counter()
for token in tokens:
    cleanToken = cleanTweet(token,exclude_re)
    terms_all = [term.lower() for term in cleanToken]
    count_all.update(terms_all)
    for w in terms_all:
        all_word.append(w)
print(count_all.most_common(100))

[('bagus', 262), ('sesuai', 186), ('barang', 176), ('pengiriman', 106), ('warna', 106), ('cepat', 103), ('bahannya', 100), (':thumbs_up:', 93), ('bahan', 91), ('terimakasihh', 80), ('suka', 74), ('adem', 55), ('warnanya', 52), ('beli', 52), ('gelap', 49), ('harga', 48), ('produk', 46), ('seller', 45), ('kasih', 43), ('terimakasih', 43), ('terima', 42), ('tp', 41), ('lembut', 40), ('zm', 36), ('gambar', 36), ('bgt', 35), ('foto', 35), ('kualitas', 33), ('tipis', 32), ('wangi', 30), ('beda', 29), ('halus', 28), ('puas', 28), ('pesanan', 27), (':smiling_face_with_heart-eyes:', 27), ('rapi', 25), ('packing', 25), ('ok', 23), ('krn', 23), ('tdk', 22), ('motif', 22), ('respon', 21), ('dgn', 21), ('order', 20), ('hijab', 20), ('dr', 20), ('jd', 20), ('jg', 20), ('deskripsi', 19), ('semoga', 19), ('penjual', 19), ('kak', 19), ('ga', 19), ('n', 18), ('diskon', 17), ('thanks', 17), ('aslinya', 17), ('sukses', 17), ('tokopedia', 17), ('promo', 17), ('kerudungnya', 17), ('barangnya', 16), ('cantik

In [ ]:
len(all_word)

5657

In [ ]:
from nlp_id.postag import PosTag
postagger = PosTag()

In [ ]:
from nlp_id.lemmatizer import Lemmatizer
lemmatizer = Lemmatizer()

In [ ]:
all_stmword = [lemmatizer.lemmatize(w) for w in all_word]

In [ ]:
print(all_stmword[:100])

['bagus', 'kerudung', 'bagus', 'barang', 'harga', 'ok', 'lembut', 'adem', 'barang', 'sesuai', 'terimakasihh', 'barang', 'original', 'bahan', 'bagus', 'keren', 'harga', 'flash', 'sale', 'thumbs up', 'light skin tone', 'thumbs up', 'light skin tone', 'thumbs up', 'light skin tone', 'fast', 'respon', 'suka', 'produk', 'zm', 'kecee', 'bangettt', 'wangi', 'barang', 'sesuai', 'terimakasihh', 'mantap', 'produk', 'puas', 'produk', 'bagus', 'klo', 'mo', 'sen', 'barang', 'kerja', 'smp', 'krn', 'gk', 'kirim', 'barang', 'bahan', 'bagus', 'realpict', 'kirim', 'barang', 'bagus', 'misan', 'senang', 'bahan', 'keren', 'n', 'adem', 'yaa', 'barang', 'bagus', 'sesuai', 'deskripsi', 'warna', 'sesuai', 'gambar', 'kirim', 'barang', 'aman', 'cepat', 'terima', 'kasih', 'bagus', 'kirim', 'aga', 'folded hands', 'so', 'cute', 'bahan', 'tipis', 'nerawang', 'untung', 'beli', 'diskon', 'kalo', 'harga', 'normal', 'gak', 'sesuai', 'bgt', 'kulitas', 'hrg', 'warna', 'gelap', 'gambar']


In [ ]:
pos = [postagger.get_pos_tag(w)[0] for w in all_stmword if len(postagger.get_pos_tag(w))>0]

In [ ]:
print(pos[:10])

[('bagus', 'JJ'), ('kerudung', 'NN'), ('bagus', 'JJ'), ('barang', 'NN'), ('harga', 'NN'), ('ok', 'NN'), ('lembut', 'JJ'), ('adem', 'NN'), ('barang', 'NN'), ('sesuai', 'JJ')]


In [ ]:
allowed_word_types = ["ADV","JJ","ADJP","VB"]
allowed_words=[w[0] for w in pos if w[1] in allowed_word_types]

In [ ]:
print(allowed_words[:100])

['bagus', 'bagus', 'lembut', 'sesuai', 'terimakasihh', 'bagus', 'sale', 'wangi', 'sesuai', 'terimakasihh', 'mantap', 'puas', 'bagus', 'kirim', 'bagus', 'kirim', 'bagus', 'senang', 'bagus', 'sesuai', 'sesuai', 'kirim', 'aman', 'cepat', 'terima', 'bagus', 'kirim', 'tipis', 'beli', 'normal', 'sesuai', 'gelap', 'kaku', 'halus', 'gelap', 'puas', 'mudah', 'disc', 'bagus', 'cantik', 'wangi', 'langsung', 'sesuai', 'terimakasihh', 'mantap', 'sesuai', 'bagus', 'sesuai', 'terimakasihh', 'bagus', 'sesuai', 'kirim', 'cepat', 'kirim', 'sesuai', 'terimakasihh', 'lebih', 'beda', 'asli', 'tipis', 'beli', 'nyari', 'bagus', 'jelek', 'mending', 'beli', 'lebar', 'enak', 'dipakenya', 'tipis', 'kirim', 'sesuai', 'skripsi', 'kirim', 'cepat', 'bagus', 'rapi', 'terima', 'neng', 'jdi', 'sesuai', 'terimakasihh', 'sesuai', 'terimakasihh', 'bagus', 'tp', 'gelap', 'kirim', 'cepat', 'bagus', 'huni', 'senang', 'bantu', 'instan', 'coba', 'tutup', 'murah', 'bagus', 'puas', 'terang']


In [ ]:
#Menyusun distribusi kata berdasarkan kemunculannya
allwords = nltk.FreqDist(allowed_words)

#Mengambil 5000 pertama dari distribusi kata berdasarkan frekwensi kemunculannya
tuple_features = list(allwords.most_common())[:1000]
word_features = [k for (k,v) in tuple_features]
#Memeriksa word_features
print(word_features[:100])

['bagus', 'sesuai', 'kirim', 'cepat', 'terimakasihh', 'beli', 'terima', 'gelap', 'tp', 'lembut', 'beda', 'wangi', 'tipis', 'puas', 'asli', 'halus', 'rapi', 'jual', 'pakai', 'kecewa', 'lumayan', 'cantik', 'enak', 'nyaman', 'ramah', 'rapih', 'mantap', 'murah', 'mudah', 'dapet', 'ukur', 'senang', 'tebal', 'tulis', 'sale', 'aman', 'cocok', 'sampe', 'hijau', 'dipake', 'cerah', 'terang', 'licin', 'harap', 'salah', 'display', 'lebar', 'dipakenya', 'selamat', 'terawang', 'langsung', 'skripsi', 'wangiiii', 'besar', 'dichat', 'buka', 'takut', 'gampang', 'biru', 'makasihh', 'mami', 'baik', 'ori', 'kasar', 'ungu', 'tua', 'tolong', 'jatuh', 'harum', 'biar', 'teliti', 'depan', 'hihiiii', 'mecca', 'cacat', 'hitam', 'discount', 'kaya', 'pilih', 'normal', 'kaku', 'disc', 'mending', 'neng', 'tutup', 'pandang', 'bingung', 'jazakallah', 'lambat', 'habis', 'bayar', 'ttp', 'khaki', 'dingin', 'aduh', 'panjang', 'medium-light', 'lepas', 'wangiiiii', 'bikin']


In [ ]:
document=[]
for w in csv:
  document.append((w[1],w[2]))

In [ ]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [ ]:
nltk.download('punkt')
featuresets = [(find_features(rev), category) for (rev, category) in document]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
print(featuresets[:5])

[({'bagus': True, 'sesuai': False, 'kirim': False, 'cepat': False, 'terimakasihh': False, 'beli': False, 'terima': False, 'gelap': False, 'tp': False, 'lembut': False, 'beda': False, 'wangi': False, 'tipis': False, 'puas': False, 'asli': False, 'halus': False, 'rapi': False, 'jual': False, 'pakai': False, 'kecewa': False, 'lumayan': False, 'cantik': False, 'enak': False, 'nyaman': False, 'ramah': False, 'rapih': False, 'mantap': False, 'murah': False, 'mudah': False, 'dapet': False, 'ukur': False, 'senang': False, 'tebal': False, 'tulis': False, 'sale': False, 'aman': False, 'cocok': False, 'sampe': False, 'hijau': False, 'dipake': False, 'cerah': False, 'terang': False, 'licin': False, 'harap': False, 'salah': False, 'display': False, 'lebar': False, 'dipakenya': False, 'selamat': False, 'terawang': False, 'langsung': False, 'skripsi': False, 'wangiiii': False, 'besar': False, 'dichat': False, 'buka': False, 'takut': False, 'gampang': False, 'biru': False, 'makasihh': False, 'mami': F

In [ ]:
# menyusun data training dan testing
trainsize = round(len(featuresets)*0.7)

random.shuffle(featuresets)
training_set = featuresets[:trainsize]
testing_set = featuresets[trainsize:]


In [ ]:
print(*testing_set[:5], sep = "\n")

({'bagus': False, 'sesuai': True, 'kirim': False, 'cepat': False, 'terimakasihh': False, 'beli': False, 'terima': False, 'gelap': False, 'tp': False, 'lembut': False, 'beda': False, 'wangi': False, 'tipis': False, 'puas': False, 'asli': False, 'halus': False, 'rapi': False, 'jual': False, 'pakai': False, 'kecewa': False, 'lumayan': False, 'cantik': False, 'enak': False, 'nyaman': False, 'ramah': False, 'rapih': False, 'mantap': False, 'murah': False, 'mudah': False, 'dapet': False, 'ukur': False, 'senang': False, 'tebal': False, 'tulis': False, 'sale': False, 'aman': False, 'cocok': False, 'sampe': False, 'hijau': False, 'dipake': False, 'cerah': False, 'terang': False, 'licin': False, 'harap': False, 'salah': False, 'display': False, 'lebar': False, 'dipakenya': False, 'selamat': False, 'terawang': False, 'langsung': False, 'skripsi': False, 'wangiiii': False, 'besar': False, 'dichat': False, 'buka': False, 'takut': False, 'gampang': False, 'biru': False, 'makasihh': False, 'mami': Fa

In [ ]:
print('\n'.join(map(str, training_set[:5])))

({'bagus': True, 'sesuai': False, 'kirim': False, 'cepat': False, 'terimakasihh': False, 'beli': False, 'terima': False, 'gelap': False, 'tp': False, 'lembut': False, 'beda': False, 'wangi': False, 'tipis': False, 'puas': False, 'asli': False, 'halus': False, 'rapi': False, 'jual': False, 'pakai': False, 'kecewa': False, 'lumayan': False, 'cantik': False, 'enak': False, 'nyaman': False, 'ramah': False, 'rapih': False, 'mantap': False, 'murah': False, 'mudah': False, 'dapet': False, 'ukur': False, 'senang': False, 'tebal': False, 'tulis': False, 'sale': False, 'aman': False, 'cocok': False, 'sampe': False, 'hijau': False, 'dipake': False, 'cerah': False, 'terang': False, 'licin': False, 'harap': False, 'salah': False, 'display': False, 'lebar': False, 'dipakenya': False, 'selamat': False, 'terawang': False, 'langsung': False, 'skripsi': False, 'wangiiii': False, 'besar': False, 'dichat': False, 'buka': False, 'takut': False, 'gampang': False, 'biru': False, 'makasihh': False, 'mami': Fa

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(25)

Classifier accuracy percent: 88.59315589353612
Most Informative Features
                    kaya = True           negati : positi =     15.7 : 1.0
               dipakenya = True           negati : positi =     15.7 : 1.0
                   lebih = True           negati : positi =     10.5 : 1.0
                    aduh = True           negati : positi =      9.4 : 1.0
                   hitam = True           negati : positi =      9.4 : 1.0
                  datang = True           negati : positi =      9.4 : 1.0
                  dingin = True           negati : positi =      9.4 : 1.0
                       i = True           negati : positi =      9.4 : 1.0
                   kecil = True           negati : positi =      9.4 : 1.0
                   khaki = True           negati : positi =      9.4 : 1.0
                    beda = True           negati : positi =      8.4 : 1.0
                      tp = True           negati : positi =      7.9 : 1.0
                   kirim = 

In [ ]:
from nltk.metrics.scores import (precision, recall)
import collections

refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)

for i, (feats, label) in enumerate(testing_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)

print('Recall:', recall(refsets['positive'], testsets['positive']))
print('Precision:', precision(refsets['positive'], testsets['positive']))

Recall: 0.9451476793248945
Precision: 0.9294605809128631


In [ ]:
#menyimpan model yang telah ditrainning ke file sav
filename = 'nb_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [ ]:
#load model classifier nb dari file
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
#melakukan prediksi data menggunakan model classifier nb yang telah diload
lsPrediksi = []
for (rev, category) in document[trainsize:]:
    result=loaded_model.classify(find_features(rev))
    lsPrediksi.append([rev, result, category])

In [ ]:
print(lsPrediksi)

[['barang oke, pengiriman lamaaa, untung admin fastrespon', 'positive', 'negative'], ['dua tiga makan ketupat', 'positive', 'positive'], ['Udah kesekian kali 👍', 'positive', 'positive'], ['warna beda kaya difoto.. lebih gelap gt kirain cerah', 'negative', 'negative'], ['Wah bagus kak kerudungnya. Packingnya juga rapi. Suka sekali. Terimakasih💕', 'positive', 'positive'], ['warna jauh lebih gelap tdk sesuai foto, agak tipis, gambarnya kurang tajam', 'negative', 'negative'], ['barang sesuai dengan pesanan dan gambar Terimakasih 🙏', 'positive', 'positive'], ['selalu suka sm produknya', 'positive', 'positive'], ['kainnya bagus, lembut dn tidak nerawang.', 'positive', 'positive'], ['Pelapak sangat responsif..kualitas dan warna okee..👍👍', 'positive', 'positive'], ['produk original, kainnya bagus halus dan adem, tapi agak nerawang sedikit, tapi oke lah. terimakasih', 'positive', 'positive'], ['Bahan wangi', 'positive', 'positive'], ['bagus banget celananya, cuma saja di adik saya pada bagian p

In [ ]:
len(lsPrediksi)

263

In [ ]:
print(*lsPrediksi, sep = "\n")

['barang oke, pengiriman lamaaa, untung admin fastrespon', 'positive', 'negative']
['dua tiga makan ketupat', 'positive', 'positive']
['Udah kesekian kali 👍', 'positive', 'positive']
['warna beda kaya difoto.. lebih gelap gt kirain cerah', 'negative', 'negative']
['Wah bagus kak kerudungnya. Packingnya juga rapi. Suka sekali. Terimakasih💕', 'positive', 'positive']
['warna jauh lebih gelap tdk sesuai foto, agak tipis, gambarnya kurang tajam', 'negative', 'negative']
['barang sesuai dengan pesanan dan gambar Terimakasih 🙏', 'positive', 'positive']
['selalu suka sm produknya', 'positive', 'positive']
['kainnya bagus, lembut dn tidak nerawang.', 'positive', 'positive']
['Pelapak sangat responsif..kualitas dan warna okee..👍👍', 'positive', 'positive']
['produk original, kainnya bagus halus dan adem, tapi agak nerawang sedikit, tapi oke lah. terimakasih', 'positive', 'positive']
['Bahan wangi', 'positive', 'positive']
['bagus banget celananya, cuma saja di adik saya pada bagian pinggang ke ke

In [ ]:
print(lsPrediksi[1][1])

positive


In [ ]:
if (lsPrediksi[1][1] == "negative"):
  print(lsPrediksi, sep = "\n")


In [ ]:
act=[]
pred=[]

for i, (feats, label) in enumerate(testing_set):
    observed = classifier.classify(feats)
    act.append(label)
    pred.append(observed)

In [ ]:
from nltk.metrics import ConfusionMatrix

In [ ]:
matrix = ConfusionMatrix(act,pred)

In [ ]:
print(matrix)

         |   n   p |
         |   e   o |
         |   g   s |
         |   a   i |
         |   t   t |
         |   i   i |
         |   v   v |
         |   e   e |
---------+---------+
negative |  <9> 17 |
positive |  13<224>|
---------+---------+
(row = reference; col = test)



In [ ]:
classifier2 = nltk.classify.SklearnClassifier(LinearSVC())
classifier2.train(training_set)

<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))>

In [ ]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier2, testing_set))*100)

Classifier accuracy percent: 58.333333333333336


In [ ]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, shuffle=False, random_state=None)

In [ ]:
for traincv, testcv in cv.split(featuresets):
    classifier = nltk.NaiveBayesClassifier.train(featuresets[traincv[0]:traincv[len(traincv)-1]])
    print ('accuracy: %.3f' % nltk.classify.util.accuracy(classifier, featuresets[testcv[0]:testcv[len(testcv)-1]]))

accuracy: 0.722
accuracy: 0.772
accuracy: 0.823
accuracy: 0.785
accuracy: 0.633


In [ ]:
from nltk.metrics.scores import (precision, recall)
import collections

In [ ]:
k=1
for traincv, testcv in cv.split(featuresets):
    training_sets = featuresets[traincv[0]:traincv[len(traincv)-1]]
    testing_sets = featuresets[testcv[0]:testcv[len(testcv)-1]]
    classifier = nltk.NaiveBayesClassifier.train(training_sets)
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)

    for i, (feats, label) in enumerate(testing_sets):
        refsets[label].add(i)
        observed = classifier.classify(feats)
        testsets[observed].add(i)
    print ("Fold - "+str(k))
    print('Recall:', recall(refsets['positive'], testsets['positive']))
    print('Precision:', precision(refsets['positive'], testsets['positive']))
    k+=1


Fold - 1
Recall: 0.975609756097561
Precision: 0.6557377049180327
Fold - 2
Recall: 0.96875
Precision: 0.6458333333333334
Fold - 3
Recall: 0.9777777777777777
Precision: 0.7719298245614035
Fold - 4
Recall: 0.9722222222222222
Precision: 0.6862745098039216
Fold - 5
Recall: 0.8636363636363636
Precision: 0.6229508196721312
